In [1]:
# Start writing code here...
!pip install nltk
!pip install gensim
!pip install xlrd
!pip install textblob


In [2]:
import numpy as np 
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import re
import string
from string import punctuation
from collections import Counter
import json

import gensim 
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

import nltk
nltk.download('punkt')

import textblob

from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings  
warnings.filterwarnings(action = 'ignore')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
#set up stop_words from nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/silviaruiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/silviaruiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/silviaruiz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
path=os.getcwd()
path

'/Users/silviaruiz/Documents/Code/Github'

In [11]:
path=path+'/Scrape_data'

In [12]:
types_reports=['10_k','10_q']

Normal LM

In [46]:
def get_ciks(type_report):
    path_to_reports=path+'/'+type_report
    ciks = os.listdir(path_to_reports)
    if '.DS_Store' in ciks: ciks.remove('.DS_Store')
    return ciks

In [51]:
ciks=[get_ciks(report) for report in types_reports]

In [54]:
ciks=ciks[0][:]
ciks

['0000816284', '0000049826']

In [5]:
def get_dates(type_report,cik):
    path_to_reports=path+'/'+type_report
    path_to_file=path_to_reports+'/'+cik+'/grabbed_text/'
    reports = os.listdir(path_to_file)
    if '.DS_Store' in reports: reports.remove('.DS_Store')
    dates=[report[11:21] for report in reports]
    return dates
    

In [82]:
reports_10k = []
for cik in ciks:  
    path_10k_files = path+'/10_k/'+cik+'/grabbed_text'
    reports_10k_temp = os.listdir(path_10k_files)
    for report in reports_10k_temp:
        #print(report)
        reports_10k.append(path_10k_files+'/'+report)
    


In [84]:
reports_10q = []
for cik in ciks: 
    path_10q_files = path+'/10_q/'+cik+'/grabbed_text'
    reports_10q_temp = os.listdir(path_10q_files)
    for report in reports_10q_temp:
        reports_10q.append(path_10q_files+'/'+report)

In [94]:
example_path = reports_10k[0]
example_path

'/Users/silviaruiz/Documents/Code/Github/Scrape_data/10_k/0000816284/grabbed_text/0000816284_2010-02-18.txt'

In [95]:
def read(path):
    f = open(path,"r")
    report_data = f.read().lower()
    return report_data

Cleaning of data

In [96]:
#1. Function to remove punctuations if a given word ended with a punctuation
def remove_punct(list1):
    list1 = [re.sub(r"[{}]+".format(punctuation), "", i) for i in list1]
    return list1

In [97]:
#2. Function to remove numbers
def remove_numbers(list1): 
    pattern = '[0-9]'
    list1 = [re.sub(pattern, '', i) for i in list1] 
    return list1

In [98]:
#3. Function for filtering words
def filter_words(string):
    return bool(re.match(r'^[a-z\']+$', string))

In [99]:
#4. Function to concatenate list of strings into one string
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += " "+str(element)
    return result

In [100]:
#5. Intersection of sets
def intersection(lst1, lst2): 
  
    # Use of hybrid method 
    temp = set(lst2) 
    lst3 = [value for value in lst1 if value in temp] 
    return lst3 

In [101]:
#7. Further cleaning (remove unwanted characters)
def clearning_characters(filtered):
    while("" in filtered) : 
        filtered.remove("") 

    while("’" in filtered) : 
        filtered.remove("’") 

    while("“" in filtered) : 
        filtered.remove("“") 

    while("”" in filtered) : 
        filtered.remove("”") 

    while("•" in filtered) : 
        filtered.remove("•") 

    while("€" in filtered) : 
        filtered.remove("€") 

    while("$" in filtered) : 
        filtered.remove("$") 
    return filtered

In [102]:
def complete_clean(path):
    data = read(path)
    word_tokens = word_tokenize(data) 
    filtered = [w for w in word_tokens if not w in stop_words] 
    filtered = remove_punct(filtered)
    filtered = remove_numbers(filtered)
    filtered = clearning_characters(filtered)
    filtered = [lemmatizer.lemmatize(word, pos = 'v') for word in filtered]
    filtered = [lemmatizer.lemmatize(word, pos = 'n') for word in filtered]
    return filtered


In [103]:
words=complete_clean(example_path)
words

['discussion',
 'analysis',
 'financial',
 'condition',
 'result',
 'operation',
 'executive',
 'summary',
 'celgene',
 'corporation',
 'subsidiary',
 'collectively',
 '\x93we\x94',
 '\x93our\x94',
 'global',
 'integrate',
 'biopharmaceutical',
 'company',
 'primarily',
 'engage',
 'discovery',
 'development',
 'commercialization',
 'innovative',
 'therapy',
 'design',
 'treat',
 'cancer',
 'immuneinflammatory',
 'relate',
 'disease',
 'primary',
 'commercial',
 'stage',
 'product',
 'include',
 'revlimid®',
 'thalomid®',
 'inclusive',
 'thalidomide',
 'celgenetm',
 'thalidomide',
 'pharmiontm',
 'subsequent',
 'acquisition',
 'pharmion',
 'corporation',
 'pharmion',
 'vidaza®',
 'alkeran®',
 'license',
 'glaxosmithkline',
 'gsk',
 'sell',
 'label',
 'march',
 'conclusion',
 'date',
 'alkeran®',
 'license',
 'gsk',
 'revlimid®',
 'oral',
 'immunomodulatory',
 'drug',
 'market',
 'unite',
 'state',
 'europe',
 'asia',
 'pacific',
 'patient',
 'multiple',
 'myeloma',
 'receive',
 'least'

Expanding LM

In [117]:
#Prepare the data for the models
def prepare_data(filtered):
    s = ' '
    text=s.join(filtered)
    data = [] 
    for i in sent_tokenize(text): 
        temp = []      
        # tokenize the sentence into words 
        for j in word_tokenize(i): 
            temp.append(j.lower()) 
    data.append(temp) 
    return data

In [118]:
data=prepare_data(words)

In [119]:
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5, sg = 2) 

In [120]:
print(os.getcwd())

/Users/silviaruiz/Documents/Code/Github


Read dictionary

In [108]:
import json

In [115]:
def count_new_words(cik,type_report,date):
    path_file=path+'/'+sector+'/'+type_report+'/'+cik+'/'+'grabbed_text'+'/'+cik+'_'+date+'.txt'
    words=complete_clean(path_file)
    data=prepare_data(words)
    #Train model on report
    model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5, sg = 2) 
    #Get positive and negative words from the report
    positive_words=dic_p_n[cik][type_report][date]['Positive']
    negative_words=dic_p_n[cik][type_report][date]['Negative']
    new_pos=[]
    new_neg=[]
    new_pos_freq=[]
    new_neg_freq=[]
    #Find similar positive and negative words from the report
    for wordz in positive_words:
        sim_words=model2.most_similar(wordz)
        for word in sim_words:
            if word[1]>0.99:  
                if word[0] not in negative_words:
                    similar=word[0]
                    new_pos.append(similar)
                    freq_p=dic_allwords[cik][type_report][date][wordz]
                    new_pos_freq.append(freq_p)
    for wordz in negative_words:
        sim_words=model2.most_similar(wordz)
        for word in sim_words:
            if word[1]>0.99:
                if word[0] not in new_pos:
                    similar=word[0]
                    new_neg.append(similar)
                    freq_n=dic_allwords[cik][type_report][date][wordz]
                    new_neg_freq.append(freq_n)
    new_pos_words_count=np.sum(new_pos_freq)
    new_neg_words_count=np.sum(new_neg_freq)
    return new_pos_words_count,new_neg_words_count#,new_pos,new_neg

In [116]:
new_pos_count,new_neg_count,new_pos,new_neg=count_new_words(cik,'10ks',dates[0])

NameError: name 'dates' is not defined

In [265]:
new_pos_count

3555

3555

In [262]:
np.sum(new_neg_freq)

3830

In [ ]:
for word

In [238]:
dic_allwords[cik]['10ks'].keys()

dict_keys(['2013-02-15', '2014-02-14', '2015-02-13', '2016-02-16', '2017-02-17', '2018-02-20'])

In [255]:
dic_allwords[cik]['10ks'][dates[0]]['financial']

368

Frequency of words

In [25]:
words_classified=pd.read_excel('Communications Words and Frequencies.xlsx')

In [26]:
number_new_pos=[]
number_new_neg=[]
for index,row in words_classified.iterrows():
    print('Working on row: ',index)
    x=str(row['Company'])
    cik='{0:0>10}'.format(x)
    type_report=row['Type']
    date=row['Date']
    new_pos_count,new_neg_count=count_new_words(cik,type_report,date)
    number_new_pos.append(new_pos_count)
    number_new_neg.append(new_neg_count)

Working on row:  0
Working on row:  1
Working on row:  2
Working on row:  3
Working on row:  4
Working on row:  5
Working on row:  6
Working on row:  7
Working on row:  8
Working on row:  9
Working on row:  10
Working on row:  11
Working on row:  12
Working on row:  13
Working on row:  14
Working on row:  15
Working on row:  16
Working on row:  17
Working on row:  18
Working on row:  19
Working on row:  20
Working on row:  21
Working on row:  22
Working on row:  23
Working on row:  24
Working on row:  25
Working on row:  26
Working on row:  27
Working on row:  28
Working on row:  29
Working on row:  30
Working on row:  31
Working on row:  32
Working on row:  33
Working on row:  34
Working on row:  35
Working on row:  36
Working on row:  37
Working on row:  38
Working on row:  39
Working on row:  40
Working on row:  41
Working on row:  42
Working on row:  43
Working on row:  44
Working on row:  45
Working on row:  46
Working on row:  47
Working on row:  48
Working on row:  49
Working on

In [297]:
words_classified

[1352]

In [28]:
words_classified['New positive words']=number_new_pos
words_classified['New negative words']=number_new_neg

In [30]:
words_classified['Positive Extended']=words_classified['Positive']+words_classified['New positive words']
words_classified['Negative Extended']=words_classified['Negative']+words_classified['New negative words']


In [32]:
words_classified['New Total Words']=words_classified['Total Words']+words_classified['New positive words']+words_classified['New negative words']

In [33]:
words_classified=words_classified.drop(columns=['Unnamed: 0', 'New positive words','New negative words'])

In [35]:
sector

'Communication'

In [37]:
words_classified.to_csv(sector+'_LM extended.csv')

In [ ]:
new_positive_10k=[]
new_negative_10k=[]


for i in range(len(dates_10ks)):
    data=prepare_data(words_10k[i])
    model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 2, sg = 2)
    positive=copy.deepcopy(positive_10k[i])
    negative=copy.deepcopy(negative_10k[i])
    new_pos=[]
    new_neg=[]
    for j in range(len(positive)):
        words=model2.most_similar(positive[j])
        for word in words:
            if word[1]>0.99:  
                if word[0] not in negative:
                    similar=word[0]
                    new_pos.append(similar)
                    new_pos_freq

    for j in range(len(negative)):
        words=model2.most_similar(negative[j])
        for word in words:
            if word[1]>0.99:
                if word[0] not in new_pos:
                    similar=word[0]
                    new_neg.append(similar)
    #positive=np.concatenate((positive, new_pos))
    #negative=np.concatenate((negative, new_neg))
    new_positive_10k.append(new_pos)
    new_negative_10k.append(new_neg)




In [ ]:
new_positive_10q=[]
new_negative_10q=[]
for i in range(len(dates_10qs)):
    print(dates_10qs[i])
    data=prepare_data(words_10q[i])
    model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 2, sg = 2)
    positive=copy.deepcopy(positive_10q[i])
    negative=copy.deepcopy(negative_10q[i])
    new_pos=[]
    new_neg=[]
    for j in range(len(positive)):
        words=model2.most_similar(positive[j])
        for word in words:
            if word[1]>0.95:  
                if word[0] not in negative:
                    similar=word[0]
                    new_pos.append(similar)
    for j in range(len(negative)):
        words=model2.most_similar(negative[j])
        for word in words:
            if word[1]>0.90:
                if word[0] not in new_pos:
                    similar=word[0]
                    new_neg.append(similar)
    #positive=np.concatenate((positive, new_pos))
    #negative=np.concatenate((negative, new_neg))
    new_positive_10q.append(new_pos)
    new_negative_10q.append(new_neg)


2014-05-08
2014-08-11
2014-11-06
2015-05-11
2015-08-07
2015-11-06
2016-05-03
2016-08-01
2016-11-01
2017-05-02
2017-08-03
2017-11-01
2018-05-03
2018-07-30
2018-10-30
2019-04-30


In [ ]:
n_positive=[(len(positive_10k[i])+len(new_positive_10k[i])) for i in range(len(positive_10k))]
n_negative=[(len(negative_10k[i])+len(new_negative_10k[i])) for i in range(len(negative_10k))]

In [ ]:
df_10k['New positive']=n_positive
df_10k['New negative']=n_negative

In [ ]:
freq_p_10k[2]

[1,
 1,
 1,
 1,
 1,
 1,
 10,
 3,
 4,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 10,
 1,
 1,
 1,
 1,
 1,
 18,
 1,
 1]

In [ ]:
positive_10k[2]

['successfully',
 'adequately',
 'able',
 'strong',
 'stronger',
 'assure',
 'gain',
 'enable',
 'enhance',
 'strongest',
 'favorable',
 'effective',
 'improvement',
 'attractive',
 'best',
 'satisfy',
 'win',
 'improve',
 'strengthen',
 'highest',
 'innovation',
 'stable',
 'greater',
 'benefit',
 'satisfaction',
 'influential']

In [ ]:
new_negative_10k[2]

['tax', 'investment', 'increase', 'acquisition', 'may', 'currency', 'interest']

In [ ]:
dates_10ks[2]

'2016-02-29'

In [ ]:
model2.most_similar('tax')

[('increase', 0.9957089424133301),
 ('million', 0.994884192943573),
 ('foreign', 0.994536280632019),
 ('expense', 0.9945088028907776),
 ('cash', 0.9941709041595459),
 ('price', 0.9941664934158325),
 ('exchange', 0.9941253066062927),
 ('rat', 0.9941070079803467),
 ('acquisition', 0.9940422773361206),
 ('include', 0.9939334392547607)]

In [ ]:
frequencies=Counter(positive_10q[0])

In [ ]:
frequencies.most_common()

[('transparency', 1),
 ('enhancement', 1),
 ('able', 1),
 ('gain', 1),
 ('enable', 1),
 ('enhance', 1),
 ('improvement', 1),
 ('attractive', 1),
 ('achieve', 1),
 ('satisfy', 1),
 ('easy', 1),
 ('strengthen', 1),
 ('highest', 1),
 ('greater', 1),
 ('benefit', 1),
 ('satisfaction', 1),
 ('influential', 1)]

In [ ]:
intersection(new_negative_10q[1],new_positive_10q[1])

[]

In [ ]:
n_positive2=[(len(positive_10q[i])+len(new_positive_10q[i])) for i in range(len(positive_10q))]
n_negative2=[(len(negative_10q[i])+len(new_negative_10q[i])) for i in range(len(negative_10q))]

In [ ]:
sum(freq_p_10q[0])

41

In [ ]:
positive_10q[0]

['transparency',
 'enhancement',
 'able',
 'gain',
 'enable',
 'enhance',
 'improvement',
 'attractive',
 'achieve',
 'satisfy',
 'easy',
 'strengthen',
 'highest',
 'greater',
 'benefit',
 'satisfaction',
 'influential']

In [ ]:
df_10q['New positive']=n_positive2
df_10q['New negative']=n_negative2

In [ ]:
df_10q

,Type,Date,N Positive Words,N Negative Words,Words,New positive,New negative
0,10q,2014-05-08,41,58,3714,17,15
1,10q,2014-08-11,48,54,3721,19,12
2,10q,2014-11-06,50,57,4017,19,13
3,10q,2015-05-11,31,56,3458,13,13
4,10q,2015-08-07,36,56,3773,15,14
5,10q,2015-11-06,37,53,3911,14,16
6,10q,2016-05-03,35,58,3683,15,13
7,10q,2016-08-01,45,56,4117,16,14
8,10q,2016-11-01,39,57,4190,12,14
9,10q,2017-05-02,40,63,3580,13,15


In [ ]:
response_df_10k=response_df[response_df.Type=='10ks']
response_df_10q=response_df[response_df.Type=='10qs']

In [ ]:
df_10k

,Type,Date,N Positive Words,N Negative Words,Words,New positive,New negative
0,10k,2014-03-06,26,27,7063,56,36
1,10k,2015-03-02,25,27,6666,35,35
2,10k,2016-02-29,26,25,6222,26,32
3,10k,2017-02-27,31,31,6581,31,41
4,10k,2018-02-23,32,35,6928,32,55
5,10k,2019-02-21,29,33,6774,39,48


In [ ]:
response_df_10k

,Company,Type,Date,Positive Return
0,TWTR,10ks,2014-03-06,-1
4,TWTR,10ks,2015-03-02,-1
8,TWTR,10ks,2016-02-29,-1
12,TWTR,10ks,2017-02-27,1
16,TWTR,10ks,2018-02-23,-1
20,TWTR,10ks,2019-02-21,1


In [ ]:
result = pd.merge(df_10k, response_df_10k, on='Date')
result2 = pd.merge(df_10q,response_df_10q, on='Date')

In [ ]:
df_test = result.append(result2).sort_values(by='Date')

df_test.iloc[:,9]

0    -1
0     1
1     1
2    -1
1    -1
3    -1
4    -1
5    -1
2    -1
6     1
7     1
8    -1
3     1
9     1
10    1
11    1
4    -1
12    1
13   -1
14   -1
5     1
15   -1
Name: Positive Return, dtype: int64

In [ ]:
from sklearn import svm
from sklearn import model_selection



train_a = 0
test_a = 0
n =100
for i in range(n):
    X = df_test.iloc[:,2:4]
    y = df_test.iloc[:,9]
    train_X,test_X,train_y,test_y = model_selection.train_test_split(X,y,random_state=i)

    clf = svm.SVC()
    clf.fit(train_X, train_y)
    train_a += clf.score(train_X,train_y)
    test_a += sum((clf.predict(test_X)==test_y))/len(test_y)
print('train score:',train_a/n)
print('test score:',test_a/n)

train score: 0.735
test score: 0.5333333333333333


In [ ]:
X = df_test.iloc[:,2:4]
y = df_test.iloc[:,9]
train_X,test_X,train_y,test_y = model_selection.train_test_split(X,y,random_state=i)

clf = svm.SVC()
clf.fit(X, y)
clf.score(X,y)

0.7727272727272727

In [ ]:
result['Signal LM']=np.where(result['N Positive Words']>result['N Negative Words'],1,-1)
result['Signal LM extended']=np.where(result['New positive']>result['New negative'],1,-1)
result['Correctness LM']=np.where(result['Signal LM']==result['Positive Return'],1,0)
result['Correctness LM extended']=np.where(result['Signal LM extended']==result['Positive Return'],1,0)
scoreLM_10k=result['Correctness LM'].sum()/len(result['Correctness LM'])
scoreLM_extended_10k=result['Correctness LM extended'].sum()/len(result['Correctness LM extended'])


In [ ]:
result2['Signal LM']=np.where(result2['N Positive Words']>result2['N Negative Words'],1,-1)
result2['Signal LM extended']=np.where(result2['New positive']>result2['New negative'],1,-1)
result2['Correctness LM']=np.where(result2['Signal LM']==result2['Positive Return'],1,0)
result2['Correctness LM extended']=np.where(result2['Signal LM extended']==result2['Positive Return'],1,0)
scoreLM_10q=result2['Correctness LM'].sum()/len(result2['Correctness LM'])
scoreLM_extended_10q=result2['Correctness LM extended'].sum()/len(result2['Correctness LM extended'])


In [ ]:
result2

,Type_x,Date,N Positive Words,N Negative Words,Words,New positive,New negative,Company,Type_y,Positive Return,Signal LM,Signal LM extended,Correctness LM,Correctness LM extended
0,10q,2014-05-08,41,58,3714,17,15,TWTR,10qs,1,-1,1,0,1
1,10q,2014-08-11,48,54,3721,19,12,TWTR,10qs,1,-1,1,0,1
2,10q,2014-11-06,50,57,4017,19,13,TWTR,10qs,-1,-1,1,1,0
3,10q,2015-05-11,31,56,3458,13,13,TWTR,10qs,-1,-1,-1,1,1
4,10q,2015-08-07,36,56,3773,15,14,TWTR,10qs,-1,-1,1,1,0
5,10q,2015-11-06,37,53,3911,14,16,TWTR,10qs,-1,-1,-1,1,1
6,10q,2016-05-03,35,58,3683,15,13,TWTR,10qs,1,-1,1,0,1
7,10q,2016-08-01,45,56,4117,16,14,TWTR,10qs,1,-1,1,0,1
8,10q,2016-11-01,39,57,4190,12,14,TWTR,10qs,-1,-1,-1,1,1
9,10q,2017-05-02,40,63,3580,13,15,TWTR,10qs,1,-1,-1,0,0


In [ ]:
scoreLM_10q

0.4375

In [ ]:
scoreLM_10k

0.5

In [ ]:
scoreLM_extended_10q

0.4375

In [ ]:
scoreLM_extended_10k

0.5

In [ ]:
result=result.drop(columns=['Type_y', 'Company',])
result2=result2.drop(columns=['Type_y', 'Company',])

KeyError: "['Type_y' 'Company'] not found in axis"

In [ ]:
result

,Type_x,Date,N Positive Words,N Negative Words,Words,New positive,New negative,Positive Return,Signal LM,Signal LM extended,Correctness LM,Correctness LM extended
0,10k,2014-03-06,26,27,7063,5320,3384,-1,-1,1,1,0
1,10k,2015-03-02,25,27,6666,2695,2835,-1,-1,-1,1,1
2,10k,2016-02-29,26,25,6222,1820,2560,-1,1,-1,0,1
3,10k,2017-02-27,31,31,6581,2511,4182,1,-1,-1,0,0
4,10k,2018-02-23,32,35,6928,2624,6820,-1,-1,-1,1,1
5,10k,2019-02-21,29,33,6774,3744,5040,1,-1,-1,0,0


In [ ]:
result2.to_csv("Twitter for ppt.csv")

In [ ]:
result.to_csv("LM and LM-extended 10k "+str(cik)+".csv")

In [ ]:
result2.to_csv("LM and LM-extended 10q "+str(cik)+".csv")